In [1]:
!pip install -q transformers datasets accelerate bitsandbytes

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 45.6 MB/s eta 0:00:00


In [4]:
# Load a portion of CNN/DailyMail dataset (25k articles)
import pandas as pd
data = pd.read_csv("train.csv", on_bad_lines='skip', engine='python')
bad = data['article'].sort_values().iloc[-9:].index
data = data.drop(bad)
data['article_length'] = data['article'].apply(lambda x: len(x))
data = data[data['article_length'] < 4000]
data = data[data['article_length'] > 1000]
data

,id,article,highlights,article_length
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",1211
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...,2544
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...,3181
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,3439
5,0004306354494f090ee2d7bc5ddbf80b63e80de6,He's been accused of making many a fashion fau...,Prime Minister and his family are enjoying an ...,1875
...,...,...,...,...
183478,79aabb0eaefc8131cee138bd7096c3cd9626730f,(CNN) -- A Chinese investor who had just bough...,Chinese investor Lam Kok is feared dead in a h...,1428
183479,79abcd645db6e80216c5d7e5df3e0b0ed03c92b8,By . Daily Mail Reporter . PUBLISHED: . 20:37 ...,Two men driving down a three-lane road in Capa...,2947
183481,79ac267194fe91b5348cc5d2153f6e7983954c57,(CNN) -- Thailand deported about 450 Hmong ref...,"Thailand deports 450 Hmong refugees to Laos, p...",1158
183482,79ac2ab39b2837e20605a9b1060820cf9b5c195b,(CNN) -- The show will go on. Ringling Bros. a...,Nine performers were injured Sunday when what'...,1081


In [ ]:
check = data['article'].apply(lambda x: x if " . " in x else "")
check.value_counts()

,count
article,
,4663
"WASHINGTON (CNN) -- CIA interrogators used waterboarding at least 266 times on two top al Qaeda suspects, according to a Bush-era Justice Department memo released by the Obama administration. Khalid Sheikh Mohammed, seen in a December sketch, was waterboarded 183 times in a month, a memo says. The controversial technique that simulates drowning -- and which President Obama calls torture -- was used at least 83 times in August 2002 on suspected al Qaeda leader Abu Zubaydah, according to the memo. Interrogators also waterboarded Khalid Sheikh Mohammed 183 times in March 2003. Mohammed is believed to be the mastermind behind the September 11, 2001, terrorist attacks on the United States. Obama released the memo Thursday, saying that ""exceptional circumstances surround these memos and require their release."" Watch other tactics outlined in memos » . The memo, dated May 30, 2005, was from then-Deputy Assistant Attorney General Steven G. Bradbury to John Rizzo, who was acting general counsel for the CIA. It paints a different picture from the one described by former CIA officer John Kiriakou. In a December 2007 interview with CNN, Kiriakou said Zubaydah had been waterboarded for ""about 30 seconds, 35 seconds"" and agreed to cooperate with interrogators the following day. In an interview on ""Fox News Sunday,"" Michael Hayden, who directed the CIA from 2006 to 2009, was asked about the number of times Mohammed was waterboarded. Hayden denounced the release of the memos and did not comment on the number, saying it was his understanding that the frequency of waterboarding was among the operational details that had not been declassified. Watch one expert say tactics 'worse than Abu Ghraib' » . The 2005 memo refers to a letter that had contained the numbers as well. Part of the reference to the letter was redacted in the released memo. Waterboarding is among the interrogation tactics that Obama has prohibited through an executive order. The CIA also has admitted waterboarding Abd al-Rahim al-Nashiri, the first person charged in the United States for the 2000 attack on the USS Cole in Yemen that killed 17 U.S. sailors. Obama said last week he felt comfortable releasing the classified memos because the Bush administration acknowledged using some of the practices associated with the memos, and the interrogation techniques were widely reported and have since been banned. ""Withholding these memos would only serve to deny facts that have been in the public domain for some time,"" Obama said in a statement. ""This could contribute to an inaccurate accounting of the past, and fuel erroneous and inflammatory assumptions about actions taken by the United States."" The president applauded the work of the U.S. intelligence community and said no one who ""carried out their duties relying in good faith upon legal advice from the Department of Justice"" would be prosecuted.",2
"(CNN) -- Basketball commentator and former hoops star Charles Barkley was arrested Wednesday in Scottsdale, Arizona, on suspicion of drunken driving, a police spokesman said. Charles Barkley was driving an Infiniti SUV through a trendy area of Scottsdale on Wednesday, police say. Barkley issued a brief statement, saying, ""I am disappointed that I put myself in that situation. The Scottsdale police were fantastic. I will not comment any further as it is a legal matter."" Lt. Eric Shuhandler of the Gilbert Police Department said an officer pulled Barkley over after he ran a stop sign in Scottsdale's Old Town area, a trendy spot known for its nightclubs and bars. Gilbert and Scottsdale are in the Phoenix metro area. ""The officer identified the driver of the 2005 Infiniti as Charles Barkley,"" according to a written statement from police. ""Mr. Barkley was administered the standard field sobriety tests after the odor of intoxicating liquor was detected."" Watch: Barkley 'disappointed' by arrest » . At a news conference later in the day, Shuhandler said B

In [5]:
import re

def clean_article(text):
    text = text.strip()

    # Check trigger condition
    if text.startswith("By .") or text[:20].startswith("(CNN) --") or text[:30].lower().startswith("by .") or text[:30].lower().find("(cnn) --") != -1:
        # Remove (CNN) -- prefix if present
        text = re.sub(r'^\s*([A-Z\s]+)?\(?CNN\)?\s*--\s*', '', text)

        # Find first actual sentence start
        match = re.search(r'\.\s+(The|A|An|In|On|At|For|It|This|That|He|She|They|There|As|After|When|While|I|We|You|One)\b', text)
        if match:
            text = text[match.start() + 2:].strip()

    # Remove "Follow @@username" if it appears
    text = re.sub(r'Follow\s+@@?\w+.*', '', text, flags=re.IGNORECASE)

    # Remove excess dots and pipes
    text = re.sub(r'(\s*\.\s*){2,}', '. ', text)
    text = re.sub(r'\|\s*', '', text)

    return text.strip()


In [ ]:
data['article'].loc[5]

"He's been accused of making many a fashion faux pas while on holiday. But the Prime Minister seems to be deaf to his critics. Yesterday David Cameron was seen in the same pair of beige loafers he wore on holiday last year. Mr Cameron, who is in Lanzarote with his family, got\xa0 the £20.99 shoes from high street store Aldo and took them with him to Portugal last summer. Retread: David Cameron with Samantha yesterday. And yes - he's wearing the same shoes . David Cameron and Samantha in Portugal last year - where he debuted his beige loafers . Yesterday he teamed them with a casual . navy blue shirt and beige shorts on a trip to Teguise in the centre of . the island with wife Samantha. As . ever fashion consultant Mrs Cameron trumped her husband in the style . stakes, wearing an elegant black maxi dress and emerald green cardigan. The . couple and their children Nancy, Arthur and Florence are spending six . days on the island in a £200-a-night restored 18th century farmhouse, . away fr

In [6]:
data['article'].apply(clean_article)

,article
0,The bishop of the Fargo Catholic Diocese in No...
1,A criminal complaint unsealed in U.S. District...
3,After initially slapping visa restrictions and...
4,Fleetwood are the only team still to have a 10...
5,He's been accused of making many a fashion fau...
...,...
183478,"On Friday, Kok introduced himself to all the e..."
183479,"The driver and passenger, not knowing what to ..."
183481,The forced repatriation prompted concern from ...
183482,"On Sunday, during a show in Providence, Rhode ..."


In [8]:
train_data = data[:100000]
train_data = train_data[['article']]

In [9]:
from datasets import Dataset
dataset = Dataset.from_pandas(train_data, preserve_index=False)

In [10]:
def clean_batch(batch):
    batch["article"] = [clean_article(a) for a in batch["article"]]
    return batch

dataset = dataset.map(clean_batch, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [12]:
dataset

Dataset({
    features: ['article'],
    num_rows: 100000
})

In [15]:
def keep_article(example):
    return {"text": example["article"].strip()}

dataset = dataset.map(keep_article, remove_columns=dataset.column_names)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [13]:
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # OPT needs this

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [16]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [17]:
block_size = 1024

def group_texts(examples):
    concatenated = sum(examples["input_ids"], [])
    total_length = len(concatenated)
    total_length = (total_length // block_size) * block_size
    result = {
        "input_ids": [concatenated[i:i + block_size] for i in range(0, total_length, block_size)],
        "attention_mask": [[1] * block_size] * (total_length // block_size),
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [18]:
split = lm_dataset.train_test_split(test_size=0.05)
train_dataset = split["train"]
eval_dataset = split["test"]

In [19]:
model = AutoModelForCausalLM.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [20]:
training_args = TrainingArguments(
    output_dir="./opt125m-news",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    num_train_epochs=5,
    fp16=torch.cuda.is_available(),
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    report_to="tensorboard"
)


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

Step,Training Loss,Validation Loss
500,3.142200,3.076531
1000,3.104400,3.044209
1500,3.065900,3.011576
2000,2.921000,2.997066
2500,2.928500,2.975102
3000,2.916300,2.952853
3500,2.771600,2.946278


Step,Training Loss,Validation Loss
500,3.142200,3.076531
1000,3.104400,3.044209
1500,3.065900,3.011576
2000,2.921000,2.997066
2500,2.928500,2.975102
3000,2.916300,2.952853
3500,2.771600,2.946278
4000,2.774600,2.932582
4500,2.776700,2.914433


KeyboardInterrupt: 

In [22]:
prompt = "The US government announced new regulations today"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

output = model.generate(
    input_ids,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.9,
    top_p=0.95
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

The US government announced new regulations today that will stop paying the fine of $500,000 to someone who has been convicted of a crime before. It will end the Department of Justice's $1.2 million fine imposed on people who have been convicted of a crime. The penalty will be in effect from January 1. New York City Mayor Michael Bloomberg said the change is to ease the burden on the city. 'It's a big change to have a zero-tolerance policy for anyone who comes forward with a crime and we think it's a huge opportunity for New York,' he told reporters in a news conference. The NYPD said it was looking into allegations that a group of at least two dozen people who were on vacation between 2007 and 2011 had sex with a tourist between 2004 and


In [23]:
model_name = "facebook/opt-125m"
tokenizer2 = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer2.pad_token = tokenizer.eos_token  # OPT needs this
model2 = AutoModelForCausalLM.from_pretrained(model_name)

prompt = "The US government announced new regulations today"
input_ids = tokenizer2(prompt, return_tensors="pt").input_ids.to(model2.device)

output = model2.generate(
    input_ids,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.9,
    top_p=0.95
)

print(tokenizer2.decode(output[0], skip_special_tokens=True))

The US government announced new regulations today, which will bar people who are working on a project in the Middle East and North Africa from participating in U.S. foreign investment.

Under the new regulations, companies in the Middle East will be barred from investing in any project that is deemed to be an illegal U.S. project, unless such investment is made by a United States-based entity.

"The rules are not only in violation of the United Nations Convention on the Law of the Sea, but also will impose significant sanctions on American companies involved in projects that violate international law," said Acting U.S. Attorney for the Southern District of New York Jason Miller.

According to the rules, companies in the Middle East and North Africa are prohibited from investing in
